In [1]:
import math
import warnings
import numpy as np
import pandas as pd
from erfa import ErfaWarning

from utils_misc import reduce_targets, compare_targets, show_unique_items
from utils_wsi_epoch_prop import wsi_J2016_prop
from utils_proper_motion import total_pm

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=ErfaWarning)

# Goal
For stars in the WSI:
1. How many primaries have mags and proper motions in the wds?
2. How many secondaries have mags and proper motions in the wds?
3. Propagate positions of stars to J2016 for Gaia query.

### A note on column names
I've renamed columns in both the wds and wsi to keep better track of which data point belongs to which catalog.

Columns in the wds always start with "wds". For example, the first few are "wds_id", "wds_dd", "wds_comp", and so on. Similarly, the wsi would be "wsi_id", "wsi_dd", "wsi_comp"...

In [2]:
# read in data
# replace empty coords with AB
wsi = pd.read_csv('data/wsi24.csv').replace({'wds_comp':np.NaN},'AB')
wds = pd.read_csv('data/wds.summ.csv').replace({'wds_comp':np.NaN},'AB')

# flag remaining nans and empty mag measurements to be located ('.' in wds)
wds = wds.replace({'wds_notes':np.NaN}, '') # dont flag empty entries in wds notes
wds = wds.replace([np.NaN, '.'],'%')

# force column data types as needed
wsi = wsi.astype( {'wds_id':str, 'wds_comp':str} )
wds = wds.astype( {'wds_id':str, 'wds_comp':str, 
                   'wds_mag1':str, 'wds_mag2':str,
                   'wds_coord':str} )

In [3]:
# remove target with no precise coord
wsi = wsi.drop(wsi.loc[ wsi.wds_id == '22267+6011' ].index)
wds = wds.drop(wds.loc[ wds.wds_id == '22267+6011' ].index)

In [4]:
# filter wsi
wsi = wsi.drop( columns=['wsi_ra_deg', 'wsi_dec_deg'] ) # drop RA and Dec columns
# wsi = wsi[ wsi['wsi_filter'] == 'y' ] # mask off all filters except y
wsi = wsi.loc[ ~wsi['wsi_dm'].isin([0.0,7.5]) ] # mask off any entry with a dm of 0.0 or 7.5 (pipeline error)
wsi = wsi.reset_index(drop=True)

In [5]:
# reduce wds to targets found in wsi
wds = reduce_targets(wds, wsi)
print( 'are unique targs the same for wds & wsi? how many are there?', compare_targets(wsi, wds) )
print( 'wsi length', len(wsi) )
print( 'wds length', len(wds) )

are unique targs the same for wds & wsi? how many are there? (True, 2137)
wsi length 2877
wds length 3809


In [6]:
# search for direct comp match between wsi and wds entry
# we should find a direct match for each, flag it if we dont
comps = []

for i, wds_id in enumerate( wsi['wds_id'] ): # loop through wds_ids in our wsi catalog

    # define the component for the current entry
    component = wsi.wds_comp.iloc[i]
    
    # find the primary match
    match = wds.loc[ (wds['wds_id']==wds_id) & (wds['wds_comp']==component) ]

    # try to add the component of the match to the list
    try:
        comps.append( match.wds_comp.iloc[0] )
    except:
        comps.append( '%' )

# check for number of flagged comps
comps.count('%')

0

now we can loop through the wsi, (knowing we get a component match for each observation) and add data from the wds to the wsi.

every observation will have a precise coordinate for the primary. They also _should_ have a mag and pm for the primary. we'll see how many secondaries have mags and pms

In [7]:
primary_coords = []
primary_mags = []
secondary_mags = []
primary_pms_ra = []
primary_pms_dec = []
secondary_pms_ra = []
secondary_pms_dec = []
wds_notes = []

for i, wds_id in enumerate( wsi['wds_id'] ): # loop through wds_ids in our wsi catalog

    # define component for current entry
    component = wsi.wds_comp.iloc[i]

    # find match based on current wds id and component
    match = wds.loc[ (wds['wds_id']==wds_id) & (wds['wds_comp']==component) ]

    # add precise primary coordinate
    primary_coords.append( match.wds_coord.iloc[0] )

    # add primary mag
    primary_mags.append( match.wds_mag1.iloc[0] )

    # add secondary mag
    secondary_mags.append( match.wds_mag2.iloc[0] )

    # add primary pms
    primary_pms_ra.append( match.wds_pm1_ra.iloc[0]  )
    primary_pms_dec.append( match.wds_pm1_dec.iloc[0] )
    
    # add secondary pms
    secondary_pms_ra.append( match.wds_pm2_ra.iloc[0] )
    secondary_pms_dec.append( match.wds_pm2_dec.iloc[0] )

    # add notes
    wds_notes.append( match.wds_notes.iloc[0] )

# add the lists to the wsi dataframe
wsi['wds_coord1'] = primary_coords
wsi['wds_mag1'] = primary_mags
wsi['wds_mag2'] = secondary_mags
wsi['wds_pm1_ra'] = primary_pms_ra
wsi['wds_pm1_dec'] = primary_pms_dec
wsi['wds_pm2_ra'] = secondary_pms_ra
wsi['wds_pm2_dec'] = secondary_pms_dec
wsi['wds_notes'] = wds_notes

In [8]:
# sanity check: compare the wds ids to the precise coordinates we just added
# the wds id should be a rounded version of the precise coords
print( wsi.wds_id.iloc[0], ':', wsi.wds_coord1.iloc[0] )
print( wsi.wds_id.iloc[-1], ':', wsi.wds_coord1.iloc[-1] )

18124-0718 : 18 12 24.02 -07 17 49.1
13203+6938 : 13 20 15.93 +69 38 08.3


now we can check for any missing mags and proper motions for the primaries and secondaries

In [9]:
# are we missing any primary mags?
list(wsi.wds_mag1).count('%')

0

all there. now we'll try secondaries

In [10]:
# are we missing any secondary mags?
list(wsi.wds_mag2).count('%')

1

we're missing one secondary mag. lets just get rid of that entry so it doesn't give us issues later

In [11]:
# drop row with missing secondary mag
wsi = wsi.drop( wsi.loc[ wsi['wds_mag2'] == '%' ].index ).reset_index(drop=True)

now lets check proper motions for the primary and secondary

In [12]:
# check primaries
print( list(wsi.wds_pm1_ra).count('%') )
print( list(wsi.wds_pm1_dec).count('%') )

2
2


In [13]:
wsi.loc[ wsi.wds_pm1_ra=='%' ]

,wds_id,wds_dd,wds_comp,wsi_date,wsi_filter,wsi_sep,wsi_sep_e,wsi_pa,wsi_pa_e,wsi_dm,...,sb_flg1,sb_flg2,wds_coord1,wds_mag1,wds_mag2,wds_pm1_ra,wds_pm1_dec,wds_pm2_ra,wds_pm2_dec,wds_notes
1230,02478+4700,J 882AB,AB,2020.8831,C,4.4171,0.0164,29.11,0.21,0.7,...,.,.,02 47 49.08 +47 01 18.6,9.8,9.8,%,%,56.0,-25.0,
2632,11563+3527,STT 241,AB,2021.2446,y,1.7945,0.0023,147.56,0.07,2.2,...,.,.,11 56 17.24 +35 26 53.3,6.82,8.74,%,%,-85.0,-23.0,O


it's odd that we have a pm for the secondary but not the primary. I'll just drop the target

In [14]:
# drop primary with no pm data
wsi = wsi.drop( wsi.loc[ wsi['wds_pm1_ra'] == '%' ].index ).reset_index(drop=True)

In [15]:
# check secondaries
# ra and dec counts should match
print( list(wsi.wds_pm2_ra).count('%') )
print( list(wsi.wds_pm2_dec).count('%') )

483
483


In [16]:
# length of wsi
len(wsi)

2874

# Conclusion
For the 2874 wsi 2024 targets:
* All primaries have magnitudes.
* All but one secondaries have magnitudes.
* All but two primaries have proper motions.
* 483 secondaries do not have proper motions.
* 83% of wsi targets have mags and proper motions for primary and secondary stars.

This sample does __NOT__ have  targets with a small separation or magnitude > 3 filtered out, which we might need to do to query gaia.

In [17]:
# force mags into float
wsi = wsi.astype({'wds_mag1':float, 'wds_mag2':float})

# remove targets brighter than mag 3
wsi = wsi.loc[ wsi['wds_mag1'] > 3.0 ]
wsi = wsi.loc[ wsi['wds_mag2'] > 3.0 ]

len(wsi) # check new length

2868

# Proper motion and epoch propagation

In [18]:
# replace flags
wsi = wsi.replace( {'wds_pm2_ra':'%', 'wds_pm2_dec':'%'}, np.nan )

# force data types
wsi = wsi.astype({'wds_pm1_ra':float, 'wds_pm1_dec':float,
                  'wds_pm2_ra':float, 'wds_pm2_dec':float}).reset_index(drop=True)

In [19]:
flags = []

# if the secondary is missing proper motion data, substitute the primary's pm and flag it
for i in range( len(wsi) ):

    # is the sec pm data a nan?
    if math.isnan( wsi.wds_pm2_ra.iloc[i] ):

        # substitute primary pm
        wsi.at[i, 'wds_pm2_ra']  = wsi.wds_pm1_ra.iloc[i]
        wsi.at[i, 'wds_pm2_dec'] = wsi.wds_pm1_dec.iloc[i]
        flags.append('!')
    
    # flag that is has its own pm
    else:
        flags.append('.')

wsi['epoch_prop_flag'] = flags

Before calculating total proper motion and propagating the positions, we need to correct the proper motions for the truncated values in the wds.

In [20]:
# which indexes have a P flag?
P_flag = list( wsi['wds_notes'].str.contains('P') )

In [21]:
# correct these proper motions
for i in range( len(wsi) ):
    if P_flag[i] == True:
        wsi.at[i, 'wds_pm1_ra']  = wsi.wds_pm1_ra.iloc[i] * 10
        wsi.at[i, 'wds_pm1_dec'] = wsi.wds_pm1_dec.iloc[i] * 10
        wsi.at[i, 'wds_pm2_ra']  = wsi.wds_pm2_ra.iloc[i] * 10
        wsi.at[i, 'wds_pm2_dec'] = wsi.wds_pm2_dec.iloc[i] * 10

In [22]:
# calculate total proper motions and add to wsi
wsi['wds_pm1'] = total_pm( wsi.wds_pm1_ra, wsi.wds_pm1_dec )
wsi['wds_pm2'] = total_pm( wsi.wds_pm2_ra, wsi.wds_pm2_dec )

In [23]:
# propogate proper motions
wsi = wsi_J2016_prop( wsi )

In [24]:
# force data types
wsi = wsi.astype({'wds_mag1':float, 'wds_mag2':float,
                  'wds_pm1':float, 'wds_pm2':float}).reset_index(drop=True)

# wsi.to_csv('data/wsi24.prepped.csv', index=False)

In [25]:
len(wsi)

2868